In [7]:
import backtest_functions as btf
import requests

In [8]:
server_time_endpoint = '/paper/api/bnf/servertime'
get_last_tick_endpoint = "/paper/api/bnf/backtest/last-tick"
place_order_endpoint = "/paper/api/bnf/orders/place"

get_last_tick_headers = {
   'X-BOT':'X-BOT',
    'sign':'sign'
}

In [10]:
btf.start_backtest('BTCUSDT')

In [12]:
import numpy as np
import time

symbol = "BTCUSDT"
timeframe = "1h"
length = 6

# Initialize variables
position = 0.0
orders = []
tick_data = []
last_tick_time = 0

while True:
    try:
        tick = btf.get_request(get_last_tick_endpoint, headers=get_last_tick_headers)

        if tick['timestamp'] - last_tick_time >= 55:
            tick_data.append(tick)

            if len(tick_data) > length:
                tick_data = tick_data[-length:]

        if len(tick_data) > length:
            tick_data = tick_data[-length:]

        closes = np.array([float(tick['ask']) for tick in tick_data])
        ma = round(closes.mean())

        upper_bound = round(ma * 1.0001, 4)
        lower_bound = round(ma * 1, 4)

        price = float(tick['ask'])
        print(price)

        if price <= round(lower_bound) and position == 0.0:
            print('buy buy buy')
            limitprice = str(closes[-1] * 1.01)
            btf.place_order("BUY", 1.1)

            position = 1.0

        elif price >= round(upper_bound) and position == 1.0:
            print('sell sell sell')
            limitprice = str(closes[-1] * 0.99)
            btf.place_order("SELL", 1)

            position = 0.0

        time.sleep(20)

    except Exception as e:
        print(f"Error: {e}")

    except KeyboardInterrupt:
        # Handle Keyboard Interrupt gracefully
        print("Keyboard Interrupt detected. Exiting...")

        break

for order in orders:
    print(order)


Keyboard Interrupt detected. Exiting...


In [ ]:
btf.stop_backtest()